In [ ]:
! pip install --user https://github.com/LAL/trackml-library/tarball/master#egg=trackml-3

In [1]:
import pandas as pd
import numpy as np
from trackml.dataset import load_event


In [2]:
path = '/data/trackMLDB/train/train_1/event000001349'
hits, cells, particles, truth = load_event(path)

In [3]:
hits

,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-87.090202,-5.976440,-1502.5,7,2,1
1,2,-64.962799,-13.262800,-1502.5,7,2,1
2,3,-55.842400,2.049980,-1502.5,7,2,1
3,4,-43.374298,-4.642420,-1502.5,7,2,1
4,5,-85.454498,-7.402500,-1502.5,7,2,1
...,...,...,...,...,...,...,...
112128,112129,-890.135010,25.823000,2952.5,18,12,98
112129,112130,-875.645020,90.500801,2952.5,18,12,98
112130,112131,-975.205017,-1.288900,2952.5,18,12,98
112131,112132,-933.573975,-2.104530,2952.5,18,12,98


In [7]:
def calc_dphi(phi1, phi2):
    """Computes phi2-phi1 given in range [-pi,pi]"""
    dphi = phi2 - phi1
    dphi[dphi > np.pi] -= 2*np.pi
    dphi[dphi < -np.pi] += 2*np.pi
    return dphi

def calc_eta(r, z):
    theta = np.arctan2(r, z)
    return -1. * np.log(np.tan(theta / 2.))

def select_segments(hits1, hits2, phi_slope_max, z0_max):
    """
    Construct a list of selected segments from the pairings
    between hits1 and hits2, filtered with the specified
    phi slope and z0 criteria.

    Returns: pd DataFrame of (index_1, index_2), corresponding to the
    DataFrame hit label-indices in hits1 and hits2, respectively.
    """
    # Start with all possible pairs of hits
    keys = ['evtid', 'r', 'phi', 'z']
    hit_pairs = hits1[keys].reset_index().merge(
        hits2[keys].reset_index(), on='evtid', suffixes=('_1', '_2'))
    # Compute line through the points
    dphi = calc_dphi(hit_pairs.phi_1, hit_pairs.phi_2)
    dz = hit_pairs.z_2 - hit_pairs.z_1
    dr = hit_pairs.r_2 - hit_pairs.r_1
    phi_slope = dphi / dr
    z0 = hit_pairs.z_1 - hit_pairs.r_1 * dz / dr
    # Filter segments according to criteria
    good_seg_mask = (phi_slope.abs() < phi_slope_max) & (z0.abs() < z0_max)
    return hit_pairs[['index_1', 'index_2']][good_seg_mask]

def construct_graph(hits, layer_pairs,
                    phi_slope_max, z0_max,
                    feature_names, feature_scale):
    """Construct one graph (e.g. from one event)"""

    # Loop over layer pairs and construct segments
    layer_groups = hits.groupby('layer')
    segments = []
    for (layer1, layer2) in layer_pairs:
        # Find and join all hit pairs
        try:
            hits1 = layer_groups.get_group(layer1)
            hits2 = layer_groups.get_group(layer2)
        # If an event has no hits on a layer, we get a KeyError.
        # In that case we just skip to the next layer pair
        except KeyError as e:
            logging.info('skipping empty layer: %s' % e)
            continue
        # Construct the segments
        segments.append(select_segments(hits1, hits2, phi_slope_max, z0_max))
    # Combine segments from all layer pairs
    segments = pd.concat(segments)

    # Prepare the graph matrices
    n_hits = hits.shape[0]
    n_edges = segments.shape[0]
    X = (hits[feature_names].values / feature_scale).astype(np.float32)
    Ri = np.zeros((n_hits, n_edges), dtype=np.uint8)
    Ro = np.zeros((n_hits, n_edges), dtype=np.uint8)
    y = np.zeros(n_edges, dtype=np.float32)
    I = hits['hit_id']

    # We have the segments' hits given by dataframe label,
    # so we need to translate into positional indices.
    # Use a series to map hit label-index onto positional-index.
    hit_idx = pd.Series(np.arange(n_hits), index=hits.index)
    seg_start = hit_idx.loc[segments.index_1].values
    seg_end = hit_idx.loc[segments.index_2].values

    # Now we can fill the association matrices.
    # Note that Ri maps hits onto their incoming edges,
    # which are actually segment endings.
    Ri[seg_end, np.arange(n_edges)] = 1
    Ro[seg_start, np.arange(n_edges)] = 1
    # Fill the segment labels
    pid1 = hits.particle_id.loc[segments.index_1].values
    pid2 = hits.particle_id.loc[segments.index_2].values
    y[:] = (pid1 == pid2)
    # Return a tuple of the results
    return Graph(X, Ri, Ro, y), I

def select_hits(hits, truth, particles, pt_min=0):
    # Barrel volume and layer ids
    vlids = [(8,2), (8,4), (8,6), (8,8),
             (13,2), (13,4), (13,6), (13,8),
             (17,2), (17,4)]
    n_det_layers = len(vlids)
    # Select barrel layers and assign convenient layer number [0-9]
    vlid_groups = hits.groupby(['volume_id', 'layer_id'])
    hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer=i)
                      for i in range(n_det_layers)])
    # Calculate particle transverse momentum
    pt = np.sqrt(particles.px**2 + particles.py**2)
    # True particle selection.
    # Applies pt cut, removes all noise hits.
    particles = particles[pt > pt_min]
    truth = (truth[['hit_id', 'particle_id']]
             .merge(particles[['particle_id']], on='particle_id'))
    # Calculate derived hits variables
    r = np.sqrt(hits.x**2 + hits.y**2)
    phi = np.arctan2(hits.y, hits.x)
    # Select the data columns we need
    hits = (hits[['hit_id', 'z', 'layer']]
            .assign(r=r, phi=phi)
            .merge(truth[['hit_id', 'particle_id']], on='hit_id'))
    # Remove duplicate hits
    hits = hits.loc[
        hits.groupby(['particle_id', 'layer'], as_index=True).r.idxmin()
    ]
    #hits.groupby(['particle_id', 'layer'], as_index=False).r.idxmin()
    
    return hits

def process_event(prefix, output_dir, pt_min, n_eta_sections, n_phi_sections,
                  eta_range, phi_range, phi_slope_max, z0_max):
    # Load the data
    evtid = int(prefix[-9:])
    if len(glob.glob(os.path.join(output_dir, "*{}*".format(evtid)))) > 0:
        logging.info("Event {} is there".format(evtid))
        return

    logging.info('Event %i, loading data' % evtid)
    hits, particles, truth = trackml.dataset.load_event(
        prefix, parts=['hits', 'particles', 'truth'])

    # Apply hit selection
    logging.info('Event %i, selecting hits' % evtid)
    hits = select_hits(hits, truth, particles, pt_min=pt_min).assign(evtid=evtid)

    # Divide detector into sections
    #phi_range = (-np.pi, np.pi)
    phi_edges = np.linspace(*phi_range, num=n_phi_sections+1)
    eta_edges = np.linspace(*eta_range, num=n_eta_sections+1)
    hits_sections = split_detector_sections(hits, phi_edges, eta_edges)

    # Graph features and scale
    feature_names = ['r', 'phi', 'z']
    feature_scale = np.array([1000., np.pi / n_phi_sections, 1000.])

    # Define adjacent layers
    n_det_layers = 10
    l = np.arange(n_det_layers)
    layer_pairs = np.stack([l[:-1], l[1:]], axis=1)

    # Construct the graph
    logging.info('Event %i, constructing graphs' % evtid)
    graphs_all = [construct_graph(section_hits, layer_pairs=layer_pairs,
                              phi_slope_max=phi_slope_max, z0_max=z0_max,
                              feature_names=feature_names,
                              feature_scale=feature_scale)
              for section_hits in hits_sections]
    graphs = [x[0] for x in graphs_all]
    IDs    = [x[1] for x in graphs_all]

    # Write these graphs to the output directory
    try:
        base_prefix = os.path.basename(prefix)
        filenames = [os.path.join(output_dir, '%s_g%03i' % (base_prefix, i))
                     for i in range(len(graphs))]
        filenames_ID = [os.path.join(output_dir, '%s_g%03i_ID' % (base_prefix, i))
                     for i in range(len(graphs))]
    except Exception as e:
        logging.info(e)
    logging.info('Event %i, writing graphs', evtid)
    save_graphs(graphs, filenames)
    for ID, file_name in zip(IDs, filenames_ID):
        np.savez(file_name, ID=ID)

In [8]:
pt_min = 0
hits, particles, truth = load_event(path, parts=['hits', 'particles', 'truth'])


In [9]:

hits = select_hits(hits, truth, particles, pt_min=pt_min)
hits


,hit_id,z,layer,r,phi,particle_id
5855,21969,43.273399,0,31.501808,2.629546,4503737066323968
12968,29417,88.876297,1,72.448486,2.632461,4503737066323968
18478,35512,136.384003,2,115.919724,2.619784,4503737066323968
23539,41520,201.632996,3,171.849625,2.613945,4503737066323968
28911,70209,315.600006,4,260.573151,2.637554,4503737066323968
...,...,...,...,...,...,...
47320,108601,726.400024,9,1016.465820,-1.605325,1035830800530022402
868,16917,-202.362000,0,32.049248,-0.533705,1035832999536492544
8393,24639,-379.829987,1,71.214142,-1.008241,1035832999536492544
906,16956,-194.220001,0,32.044922,0.026701,1035833068255969280


In [38]:
output = nhits.groupby(['particle_id', 'layer'], as_index=True).r.idxmin()
output

particle_id          layer
4503737066323968     0         5855
                     1        12968
                     2        18478
                     3        23539
                     4        28911
                              ...  
1035830800530022402  9        47320
1035832999536492544  0          868
                     1         8393
1035833068255969280  0          906
                     1         8454
Name: r, Length: 38412, dtype: int64

In [39]:
output = nhits.loc[nhits.groupby(['particle_id', 'layer'], as_index=True).r.idxmin()]
output

,hit_id,z,layer,r,phi,particle_id
5855,21969,43.273399,0,31.501808,2.629546,4503737066323968
12968,29417,88.876297,1,72.448486,2.632461,4503737066323968
18478,35512,136.384003,2,115.919724,2.619784,4503737066323968
23539,41520,201.632996,3,171.849625,2.613945,4503737066323968
28911,70209,315.600006,4,260.573151,2.637554,4503737066323968
...,...,...,...,...,...,...
47320,108601,726.400024,9,1016.465820,-1.605325,1035830800530022402
868,16917,-202.362000,0,32.049248,-0.533705,1035832999536492544
8393,24639,-379.829987,1,71.214142,-1.008241,1035832999536492544
906,16956,-194.220001,0,32.044922,0.026701,1035833068255969280


In [ ]:
in_idxs = [tuple(v) for v in in_idxs]